**casual model run**

In [1]:
import warnings

In [3]:
#remove warnings
warnings.filterwarnings('ignore')

In [44]:
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline

In [ ]:
# for tokenization
tokenizer=AutoTokenizer.from_pretrained('gpt2')

In [ ]:
#for model
model=AutoModelForCausalLM.from_pretrained(
    'gpt2',
    device_map='cpu',
    torch_dtype='auto', #data type auto
    trust_remote_code=True, # trust the repository code
)

In [ ]:
#pipelining the model and tokenizer in one pipeline
generator=pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,#no randomness, deterministic

)

In [ ]:

prompt="write deep psycho love message to my girl-friend"
output=generator(prompt)

In [13]:
print(output[0]['generated_text']) #only return the response not the prompt

.

"You know why you're jealous of me?"

"Because…" He pauses, his face turning bright red. "I love you! You're my love and my friend."

"You really should be jealous of me


**Model Analyze**

In [14]:
model #peeking inside engine

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
model.transformer.wte #embed text vectors; model.model.embed_tokens for other models

Embedding(50257, 768)

In [19]:
model.transformer.h[0] #first transformer block

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [46]:
#model.model for all layers
#model.model.layers[0] for first transformer block
model.lm_head #final layer

Linear(in_features=768, out_features=50257, bias=False)

**Complete the text**

In [37]:
message="The capital of France is the "

In [38]:
input_ids=tokenizer(message,return_tensors='pt').input_ids
# all message token ids

In [39]:
model_output=model.transformer(input_ids)
model_output[0].shape
# feeded into the model

torch.Size([1, 7, 768])

In [40]:
lm_head_output=model.lm_head(model_output[0])
lm_head_output.shape
# final layer output

torch.Size([1, 7, 50257])

In [41]:
ParisId=lm_head_output[0,-1].argmax(-1)
ParisId
#the highest number token or most likely after the last token in batch 0

tensor(933)

In [42]:
actualPredictedWord=tokenizer.decode(ParisId)
actualPredictedWord
#predicted text is wrong due to old model

'vern'